# OpenBabel for chemical format conversion

The software must first be installed (http://openbabel.org/wiki/Main_Page) then the python package must be installed for the bindings (https://open-babel.readthedocs.io/en/latest/UseTheLibrary/Python.html)

In [1]:
from openbabel import openbabel
from pathlib import Path
import os
import glob

In [ ]:
dir = Path("../../Ressources/2_Openbabel_conversion")

### Setting up the openbabel converter object

In [ ]:
# Get a converter object
conv = openbabel.OBConversion()

#conv.GetSupportedInputFormat() # Check input format
conv.GetSupportedOutputFormat() # Check output format


In [ ]:
def openbabel_conversion(infile, informat, outformat):
    #print(str(infile.parent / "converted" / infile.with_suffix("." + outformat).name))
    conv = openbabel.OBConversion()
    conv.SetInAndOutFormats(informat, outformat)
    print(f"Converting {infile.name} from {informat} to {outformat}")
    # put conversion to a converted folder
    outdir = Path(infile.parent / "converted")
    outdir.mkdir(parents=True, exist_ok=True)
    conv.OpenInAndOutFiles(str(infile), str(outdir / infile.with_suffix("." + outformat).name))
    conv.Convert()
    conv.CloseOutFile()
    print("Conversion done.")

In [ ]:
files = dir.glob('*')

for file in files:
    print(file)
    if file.suffix == ".smiles":
        openbabel_conversion(file, "smiles", "inchi")
        openbabel_conversion(file, "smiles", "inchikey")
    elif file.suffix == ".inchi":
        openbabel_conversion(file, "inchi", "smiles")
        openbabel_conversion(file, "inchi", "inchikey")
    elif file.suffix == ".inchikey":
        openbabel_conversion(file, "inchikey", "inchi")
        openbabel_conversion(file, "inchikey", "smiles")
    else: 
        print("No conversion for this file")

In [ ]:
for i in dir.glob("COCO*"):
    print(i)

### Conversion whole database to specific format

In [1]:
import sqlite3
import pandas as pd

DB_PATH = "../../Database/compounds.sqlite"

con = sqlite3.connect(DB_PATH)
cur = con.cursor()

In [2]:
all_smiles = pd.read_sql_query('SELECT ID, SMILES, InChI FROM compounds', con)

In [ ]:
# Trying to convert to mol format
all_smiles.to_csv('/Volumes/4TB/Users/dpflieger/tmp/all_smiles.smiles', columns = ["SMILES"], header = False, index = False)
all_smiles.to_csv('/Volumes/4TB/Users/dpflieger/tmp/all_smiles.inchi', columns = ["InChI"], header = False, index = False)

# The format

In [ ]:
conv = openbabel.OBConversion()
conv.SetInAndOutFormats("smiles", "mol")
conv.SetOptions('3', openbabel.OBConversion.OUTOPTIONS)
conv.SetOptions('gen3d', openbabel.OBConversion.OUTOPTIONS)
conv.OpenInAndOutFiles("/Volumes/4TB/Users/dpflieger/tmp/all_smiles.smiles", "/Volumes/4TB/Users/dpflieger/tmp/all_smiles.mol")
conv.Convert()
conv.CloseOutFile()

In [8]:
# RUN OPENBABEL FROM THE COMMAND LINE
import os

smiles_dir = Path("/Volumes/4TB/Users/dpflieger/Projects/Gaquerel/Elser/tmp_obabel/smiles")
smiles_dir.mkdir(parents=True, exist_ok=True)

for index, row in all_smiles.iterrows():
    #print(row['ID'], row['SMILES'], row["InChI"])
    #with open((smiles_dir / row["ID"]).with_suffix(".smiles"), "w") as out:
    #    print(row["SMILES"], file = out)    
    with open("/Volumes/4TB/Users/dpflieger/tmp/obabel_commands.txt", "a") as out2:
        print(f"/home/dpflieger/Softs/openbabel-2.4.1/build/bin/obabel /ssd_workspace/dpflieger/conversion/smiles/{row['ID']}.smiles -o mol -O /ssd_workspace/dpflieger/conversion/mol/{row['ID']}.mol -x3 --gen3d", file = out2)

# ~/Softs/openbabel/build/bin/obabel all_smiles.smiles -o mol -O test.mol -x3 --gen3d

### Convert all smiles files to .mol format

obabel command line and with option *-x3 --gen3d*

In [ ]:
import subprocess
import tqdm

mol_dir = Path("/Volumes/4TB/Users/dpflieger/Projects/Gaquerel/Elser/tmp_obabel/mol")
mol_dir.mkdir(parents=True, exist_ok=True)

#paths = [i.path for i in islice(os.scandir(path), 100))]

for i in tqdm(smiles_dir.glob("*")):
    outfile = mol_dir / str(i.name).replace(".smiles", ".mol")
    #print(outfile)
    #break
    #subprocess.call(["/Volumes/4TB/Users/dpflieger/Softs/openbabel/build/bin/obabel", i, "-o", "mol", "-O", outfile, "-x3", "--gen3d"])
    os.system(f"~/Softs/openbabel/build/bin/obabel {i} -o mol -O {outfile} -x3 --gen3d")
    

## Note: openbabel on babel node

The last version of openbabel (v3.1.1) doesn't work. There is a bug with the --gen3D option...  
I installed and tried an older version (v2.4.1) and it is working now.

example of command:

```
/home/dpflieger/Softs/openbabel-2.4.1/build/bin/obabel ZINC000000001411.smiles -o mol -O /ssd_workspace/dpflieger/conversion/mol/ZINC000000001411.mol -x3 --gen3d
```